# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import requests
# import json

# Define a function to return a list of dictionaries with information about the requested city
def citybikes_locations(city):
    citybike_networks = requests.get("https://api.citybik.es/v2/networks").json()
    list_of_dict = []
    for citybike_dict in citybike_networks['networks']:
        new_city = citybike_dict['location']['city']
        if new_city.lower() == city.lower():
            list_of_dict.append(citybike_dict)
    return list_of_dict

# Define a function to return a list of dictionaries about the city's stations
def city_stations(city):
    station_dict = citybikes_locations(city)
    if not station_dict:
        print('Error: No info found for {}'.format(city))
        return None
    network_address = station_dict[0]['href']
    url = "https://api.citybik.es/{}".format(network_address)
    return requests.get(url).json()['network']['stations']

In [2]:
# Vancouver is the only Canadian city on the citybik.es api:
van_network = citybikes_locations("Vancouver")

# Save the json string to data folder as "van_network.json"

In [3]:
# Run api query for the Vancouver citybikes stations
van_stations = city_stations("Vancouver")

# Save the json data in the data folder as "van_stations.json"

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [13]:
# Define a function that will return a pd.df about the stations for input city
def station_data(city):
    station_info = city_stations(city)
    station_list = []
    for info in station_info:
        station_details = {
              'Station Name': info['name']
            , 'slots': info['extra']['slots']
            , 'empty_slots': info['empty_slots']
            , 'free_bikes': info['free_bikes']
            , 'status': info['extra']['status']
            , 'bank_card': ', '.join(info['extra']['bank_card']) if info['extra']['bank_card'] else 'No'
            , 'latitude': info['latitude']
            , 'longitude': info['longitude']
            , 'timestamp': info['timestamp']
            , 'Unique ID': info['extra']['uid']
        }
        station_list.append(station_details)
    return pd.DataFrame(station_list)

Put your parsed results into a DataFrame.

In [5]:
# Load the data into the DataFrame  - all information gathered except for 
df = station_data('Vancouver')

In [6]:
# Lets check out the size of our new df!
df.shape

(241, 10)

In [7]:
# To display the top 5 rows and headers
df.head(5)

,Station Name,slots,empty_slots,free_bikes,status,bank_card,latitude,longitude,timestamp,Unique ID
0,0001 10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,0001
1,0004 Yaletown-Roundhouse Station,16,1,15,online,No,49.274566,-123.121817,2022-11-06T07:22:52.006000Z,0004
2,0005 Dunsmuir & Beatty,26,8,18,online,No,49.279764,-123.110154,2022-11-06T07:22:52.007000Z,0005
3,0007 12th & Yukon (City Hall),16,9,7,online,No,49.260599,-123.113504,2022-11-06T07:22:52.008000Z,0007
4,0008 8th & Ash,16,9,7,online,No,49.264215,-123.117772,2022-11-06T07:22:52.009000Z,0008


In [8]:
# To display the last 5 rows
df.tail(5)

,Station Name,slots,empty_slots,free_bikes,status,bank_card,latitude,longitude,timestamp,Unique ID
236,0516 Heather & 29th,22,21,1,online,No,49.245535,-123.120496,2022-11-06T07:22:52.164000Z,0516
237,0041 Cardero & Robson,26,0,26,online,No,49.289255,-123.132677,2022-11-06T07:22:52.025000Z,0041
238,0352 Commercial & 20th,18,15,3,online,No,49.252656,-123.067965,2022-11-06T07:22:52.152000Z,0352
239,0196 Hornby & Drake,24,9,15,online,No,49.277527,-123.129464,2022-11-06T07:22:52.105000Z,0196
240,210 Burrard & 14th,18,10,8,online,No,49.259469,-123.145718,2022-11-06T07:22:52.172000Z,210


In [9]:
# Checking the types of data
df.dtypes

Station Name     object
slots             int64
empty_slots       int64
free_bikes        int64
status           object
bank_card        object
latitude        float64
longitude       float64
timestamp        object
Unique ID        object
dtype: object

In [10]:
# Or also/alternatively:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Station Name  241 non-null    object 
 1   slots         241 non-null    int64  
 2   empty_slots   241 non-null    int64  
 3   free_bikes    241 non-null    int64  
 4   status        241 non-null    object 
 5   bank_card     241 non-null    object 
 6   latitude      241 non-null    float64
 7   longitude     241 non-null    float64
 8   timestamp     241 non-null    object 
 9   Unique ID     241 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 19.0+ KB


In [11]:
# Save a back up just incase we come to regret some of our changes:
df_backup = df.copy()

# Confirm the save:
df_backup.head(5)

,Station Name,slots,empty_slots,free_bikes,status,bank_card,latitude,longitude,timestamp,Unique ID
0,0001 10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,0001
1,0004 Yaletown-Roundhouse Station,16,1,15,online,No,49.274566,-123.121817,2022-11-06T07:22:52.006000Z,0004
2,0005 Dunsmuir & Beatty,26,8,18,online,No,49.279764,-123.110154,2022-11-06T07:22:52.007000Z,0005
3,0007 12th & Yukon (City Hall),16,9,7,online,No,49.260599,-123.113504,2022-11-06T07:22:52.008000Z,0007
4,0008 8th & Ash,16,9,7,online,No,49.264215,-123.117772,2022-11-06T07:22:52.009000Z,0008


In [12]:
df.to_csv('data/df_citybikes.csv')